## Notebook with feature engineering process

### Curated Features list (in addition to columns)

#### 1 Features based on Sessions actions:  
1. [x] Create features event_i with according to:  
    * event_i means that it's action_info event of order i  
    * take its first order in session, i.e. if events are show_nan_nan, show_view_p3 then values for show_view_p3 is 2  
    * normalize by deviding by total number of events in user's session
2. [x] COUNT for each action_type
3. [x] MEAN, MAX and other descriptive statistics of secs_elapsed deltas

#### 2 Aggregated on Sessions:  
1. [x] COUNT DISTINCT of device_type
2. [ ] % time spent on each action type
3. [ ] count sessions per each device, MODE of Device type  
4. [ ] given that timestamp_first_active is the start of the session, analyze hour (0-23) of activity

#### 3 Transformed from users:
1. [x] Hour of first activity - users['hour_factive'] = users.timestamp_first_active.dt.hour
2. [x] date of week of account_created

**TODO**: use age_gender_bktd and countries data for features generation

In [1]:
import pandas as pd
from datetime import datetime
from tqdm.notebook import tqdm
import numpy as np
from scipy import stats
from collections import Counter

pd.options.display.float_format = "{:.2f}".format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
tqdm.pandas()
%load_ext autotime

time: 296 µs (started: 2021-08-18 18:40:29 +00:00)


### 0. Loading Data

In [2]:
users = pd.read_parquet('../data/processed/users.parquet')
users.shape

(275547, 16)

time: 382 ms (started: 2021-08-18 18:40:29 +00:00)


In [3]:
sessions = pd.read_parquet('../data/processed/sessions.parquet')
sessions.shape

(10533241, 7)

time: 3.6 s (started: 2021-08-18 18:40:30 +00:00)


In [4]:
users.head()

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,train_flag
0,gxn3p5htnn,2010-06-28,2009-03-19 04:32:55,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF,1
1,820tgsjxq7,2011-05-25,2009-05-23 17:48:09,MALE,38.00,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF,1
2,4ft3gnwmtx,2010-09-28,2009-06-09 23:12:47,FEMALE,56.00,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US,1
3,bjjt8pjhuk,2011-12-05,2009-10-31 06:01:29,FEMALE,42.00,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other,1
4,87mebub9p4,2010-09-14,2009-12-08 06:11:05,-unknown-,41.00,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US,1


time: 42.7 ms (started: 2021-08-18 18:40:33 +00:00)


In [5]:
sessions.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed,action_info
0,d1mm9tcy42,lookup,None,None,Windows Desktop,319.00,lookup_nan_nan
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.00,search_results_click_view_search_results
2,d1mm9tcy42,lookup,None,None,Windows Desktop,301.00,lookup_nan_nan
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.00,search_results_click_view_search_results
4,d1mm9tcy42,lookup,None,None,Windows Desktop,435.00,lookup_nan_nan


time: 28.9 ms (started: 2021-08-18 18:40:33 +00:00)


### 2. Getting features based on Sessions

In [6]:
sessions.secs_elapsed.fillna(-1, inplace=True)
sessions.sort_values(['user_id', 'secs_elapsed'], inplace=True)
sessions.reset_index(drop=True, inplace=True)
sessions.shape

(10533241, 7)

time: 8.87 s (started: 2021-08-18 18:40:33 +00:00)


In [7]:
sessions.head(10)

,user_id,action,action_type,action_detail,device_type,secs_elapsed,action_info
0,00023iyk9l,callback,partner_callback,oauth_response,Mac Desktop,-1.00,callback_partner_callback_oauth_response
1,00023iyk9l,pending,booking_request,pending,Mac Desktop,0.00,pending_booking_request_pending
2,00023iyk9l,personalize,data,wishlist_content_update,Mac Desktop,6.00,personalize_data_wishlist_content_update
3,00023iyk9l,show,None,None,Mac Desktop,45.00,show_nan_nan
4,00023iyk9l,similar_listings,data,similar_listings,Mac Desktop,81.00,similar_listings_data_similar_listings
5,00023iyk9l,similar_listings,data,similar_listings,Mac Desktop,94.00,similar_listings_data_similar_listings
6,00023iyk9l,show,None,None,Mac Desktop,112.00,show_nan_nan
7,00023iyk9l,similar_listings,data,similar_listings,Mac Desktop,155.00,similar_listings_data_similar_listings
8,00023iyk9l,index,view,view_search_results,Mac Desktop,163.00,index_view_view_search_results
9,00023iyk9l,show,view,p3,Mac Desktop,182.00,show_view_p3


time: 21.8 ms (started: 2021-08-18 18:40:42 +00:00)


### 2.1 Generating features based on action_info Events vs its Order in the session stream with Normalization

Sessions actions:  
Create features event_i with according to:  
    * event_i means that it's action_info event of order i  
    * take its first order in session, i.e. if events are show_nan_nan, show_view_p3 then values for show_view_p3 is 2  
    * normalize by deviding by total number of events in user's session  

### Small change to previous approaches, taking only those action_info, which are present in both train and test datasets, without dropping the rest of action_info

In [8]:
# actions_info = list(sessions.action_info.unique())
# len(actions_info)
actions_info = set(sessions[sessions.user_id.isin(users[users.train_flag == 1].id)].action_info.unique())
actions_info = actions_info.intersection(set(sessions[sessions.user_id.isin(users[users.train_flag == 0].id)].action_info.unique()))
actions_info = list(actions_info)
len(actions_info)

336

time: 4.64 s (started: 2021-08-18 18:40:42 +00:00)


In [ ]:
# actions_info = list(sessions.action_info.unique())
# len(actions_info)
action_details = set(sessions[sessions.user_id.isin(users[users.train_flag == 1].id)].action_detail.unique())
action_details = action_details.intersection(set(sessions[sessions.user_id.isin(users[users.train_flag == 0].id)].action_detail.unique()))
action_details = list(action_details)
len(action_details)

127

time: 3.97 s (started: 2021-08-18 18:40:47 +00:00)


In [ ]:
sessions.action_detail.nunique()

155

time: 732 ms (started: 2021-08-18 18:40:51 +00:00)


In [ ]:
tmp = sessions[['user_id', 'action_info']].groupby('user_id', as_index=False).agg(list)
tmp.shape

(135483, 2)

time: 5.24 s (started: 2021-08-18 18:40:52 +00:00)


In [ ]:
tmp['size'] = tmp.action_info.apply(lambda x: len(x))

time: 115 ms (started: 2021-08-18 18:40:57 +00:00)


In [ ]:
tmp.head()

,user_id,action_info,size
0,00023iyk9l,"[callback_partner_callback_oauth_response, pen...",40
1,0010k6l0om,"[callback_partner_callback_oauth_response, sea...",63
2,001wyh0pz8,"[create_submit_signup, search_click_view_searc...",90
3,0028jgx1x1,"[create_submit_create_user, show_view_user_pro...",31
4,002qnbzfs5,"[campaigns_nan_nan, click_click_book_it, show_...",789


time: 19.5 ms (started: 2021-08-18 18:40:57 +00:00)


In [ ]:
tmp.columns = ['user_id', 'action_info', 'seassion_length']

time: 1.94 ms (started: 2021-08-18 18:40:57 +00:00)


In [ ]:
def find_action_info_pos(ai, ais):
    try:
        return ais.index(ai) + 1
    except ValueError:
        return None

time: 737 µs (started: 2021-08-18 18:40:59 +00:00)


In [17]:
for ai in tqdm(actions_info):
    tmp[f'ai_{ai}'] = tmp.action_info.apply(lambda x: find_action_info_pos(ai, x))

  0%|          | 0/336 [00:00<?, ?it/s]

/tmp/ipykernel_323/1504429830.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  tmp[f'ai_{ai}'] = tmp.action_info.apply(lambda x: find_action_info_pos(ai, x))


time: 2min 4s (started: 2021-08-18 18:41:13 +00:00)


In [18]:
tmp.head()

,user_id,action_info,seassion_length,ai_handle_vanity_url_-unknown-_-unknown-,ai_phone_verification_number_submitted_for_call_-unknown-_-unknown-,ai_home_safety_landing_-unknown-_-unknown-,ai_delete_submit_delete_phone_numbers,ai_search_results_click_view_search_results,ai_guarantee_view_host_guarantee,ai_track_page_view_nan_nan,ai_populate_from_facebook_-unknown-_-unknown-,ai_phone_verification_error_-unknown-_-unknown-,ai_facebook_auto_login_-unknown-_-unknown-,ai_payout_delete_-unknown-_-unknown-,ai_phone_verification_number_submitted_for_sms_-unknown-_-unknown-,ai_push_notification_callback_-unknown-_-unknown-,ai_15_message_post_message_post,ai_login_view_login_page,ai_create_submit_create_user,ai_phone_verification_call_taking_too_long_-unknown-_-unknown-,ai_destroy_-unknown-_-unknown-,ai_sync_-unknown-_-unknown-,ai_contact_new_-unknown-_-unknown-,ai_p4_terms_click_p4_terms,ai_clickthrough_-unknown-_-unknown-,ai_hospitality_-unknown-_-unknown-,ai_reviews_data_listing_reviews,ai_tos_confirm_-unknown-_-unknown-,ai_create_multiple_-unknown-_-unknown-,ai_authorize_-unknown-_-unknown-,ai_localized_-unknown-_-unknown-,ai_recommend_-unknown-_-unknown-,ai_create_view_list_your_space,ai_travel_plans_current_view_your_trips,ai_city_count_-unknown-_-unknown-,ai_home_safety_terms_-unknown-_-unknown-,ai_terms_view_terms_and_privacy,ai_payout_preferences_view_account_payout_preferences,ai_payment_instruments_data_payment_instruments,ai_pending_tickets_-unknown-_-unknown-,ai_other_hosting_reviews_-unknown-_-unknown-,ai_confirm_email_click_confirm_email_link,ai_request_new_confirm_email_click_request_new_confirm_email,ai_complete_redirect_-unknown-_-unknown-,ai_collections_view_user_wishlists,ai_identity_-unknown-_-unknown-,ai_create_-unknown-_-unknown-,ai_kba_update_-unknown-_-unknown-,ai_maybe_information_message_post_message_post,ai_delete_submit_delete_listing,ai_supported_-unknown-_-unknown-,ai_photography_update_-unknown-_-unknown-,ai_at_checkpoint_booking_request_at_checkpoint,ai_kba_-unknown-_-unknown-,ai_envoy_bank_details_redirect_-unknown-_-unknown-,ai_forgot_password_submit_forgot_password,ai_available_data_trip_availability,ai_approve_-unknown-_-unknown-,ai_itinerary_view_guest_itinerary,ai_pending_-unknown-_-unknown-,ai_payoneer_account_redirect_-unknown-_-unknown-,ai_update_submit_update_listing,ai_apply_coupon_click_click_apply_coupon_click,ai_signup_weibo_-unknown-_-unknown-,ai_edit_-unknown-_-unknown-,ai_notifications_data_notifications,ai_remove_dashboard_alert_-unknown-_-unknown-,ai_show_data_translations,ai_update_cached_data_admin_templates,ai_listings_-unknown-_-unknown-,ai_life_-unknown-_-unknown-,ai_preapproval_message_post_message_post,ai_requested_view_p5,ai_show_view_view_listing,ai_office_location_-unknown-_-unknown-,ai_ask_question_submit_contact_host,ai_reviews_data_user_reviews,ai_currencies_nan_nan,ai_signup_weibo_referral_-unknown-_-unknown-,ai_why_host_-unknown-_-unknown-,ai_domains_-unknown-_-unknown-,ai_phone_verification_phone_number_removed_-unknown-_-unknown-,ai_social_connections_data_user_social_connections,ai_ajax_refresh_subtotal_click_change_trip_characteristics,ai_recent_reservations_-unknown-_-unknown-,ai_settings_-unknown-_-unknown-,ai_jumio_-unknown-_-unknown-,ai_feed_-unknown-_-unknown-,ai_localization_settings_nan_nan,ai_click_click_book_it,ai_detect_fb_session_-unknown-_-unknown-,ai_email_by_key_-unknown-_-unknown-,ai_review_page_-unknown-_-unknown-,ai_other_hosting_reviews_first_-unknown-_-unknown-,ai_complete_status_-unknown-_-unknown-,ai_read_policy_click_click_read_policy_click,ai_listings_view_user_listings,ai_recommendations_data_listing_recommendations,ai_pay_-unknown-_-unknown-,ai_referrer_status_-unknown-_-unknown-,ai_apply_coupon_error_click_apply_coupon_error,ai_new_-unknown-_-unknown-,ai_guest_booked_elsewhere_message_post_message_post,ai_photography_-unknown-_-unknown-,ai_webcam_upload_-unknown-_-unknown-,ai_personalize_data_wishlist_content_update,ai_signup_login_view_sign

time: 347 ms (started: 2021-08-18 18:43:17 +00:00)


### 2.1.b Adding action_info counts features

In [19]:
def get_action_info_count(ai, ais):
    return ais.count(ai)

time: 610 µs (started: 2021-08-18 18:43:17 +00:00)


In [20]:
for ai in tqdm(actions_info):
    tmp[f'count_{ai}'] = tmp.action_info.apply(lambda x: get_action_info_count(ai, x))

  0%|          | 0/336 [00:00<?, ?it/s]

/tmp/ipykernel_323/2557823177.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  tmp[f'count_{ai}'] = tmp.action_info.apply(lambda x: get_action_info_count(ai, x))


time: 1min 46s (started: 2021-08-18 18:43:17 +00:00)


In [21]:
tmp.head()

,user_id,action_info,seassion_length,ai_handle_vanity_url_-unknown-_-unknown-,ai_phone_verification_number_submitted_for_call_-unknown-_-unknown-,ai_home_safety_landing_-unknown-_-unknown-,ai_delete_submit_delete_phone_numbers,ai_search_results_click_view_search_results,ai_guarantee_view_host_guarantee,ai_track_page_view_nan_nan,ai_populate_from_facebook_-unknown-_-unknown-,ai_phone_verification_error_-unknown-_-unknown-,ai_facebook_auto_login_-unknown-_-unknown-,ai_payout_delete_-unknown-_-unknown-,ai_phone_verification_number_submitted_for_sms_-unknown-_-unknown-,ai_push_notification_callback_-unknown-_-unknown-,ai_15_message_post_message_post,ai_login_view_login_page,ai_create_submit_create_user,ai_phone_verification_call_taking_too_long_-unknown-_-unknown-,ai_destroy_-unknown-_-unknown-,ai_sync_-unknown-_-unknown-,ai_contact_new_-unknown-_-unknown-,ai_p4_terms_click_p4_terms,ai_clickthrough_-unknown-_-unknown-,ai_hospitality_-unknown-_-unknown-,ai_reviews_data_listing_reviews,ai_tos_confirm_-unknown-_-unknown-,ai_create_multiple_-unknown-_-unknown-,ai_authorize_-unknown-_-unknown-,ai_localized_-unknown-_-unknown-,ai_recommend_-unknown-_-unknown-,ai_create_view_list_your_space,ai_travel_plans_current_view_your_trips,ai_city_count_-unknown-_-unknown-,ai_home_safety_terms_-unknown-_-unknown-,ai_terms_view_terms_and_privacy,ai_payout_preferences_view_account_payout_preferences,ai_payment_instruments_data_payment_instruments,ai_pending_tickets_-unknown-_-unknown-,ai_other_hosting_reviews_-unknown-_-unknown-,ai_confirm_email_click_confirm_email_link,ai_request_new_confirm_email_click_request_new_confirm_email,ai_complete_redirect_-unknown-_-unknown-,ai_collections_view_user_wishlists,ai_identity_-unknown-_-unknown-,ai_create_-unknown-_-unknown-,ai_kba_update_-unknown-_-unknown-,ai_maybe_information_message_post_message_post,ai_delete_submit_delete_listing,ai_supported_-unknown-_-unknown-,ai_photography_update_-unknown-_-unknown-,ai_at_checkpoint_booking_request_at_checkpoint,ai_kba_-unknown-_-unknown-,ai_envoy_bank_details_redirect_-unknown-_-unknown-,ai_forgot_password_submit_forgot_password,ai_available_data_trip_availability,ai_approve_-unknown-_-unknown-,ai_itinerary_view_guest_itinerary,ai_pending_-unknown-_-unknown-,ai_payoneer_account_redirect_-unknown-_-unknown-,ai_update_submit_update_listing,ai_apply_coupon_click_click_apply_coupon_click,ai_signup_weibo_-unknown-_-unknown-,ai_edit_-unknown-_-unknown-,ai_notifications_data_notifications,ai_remove_dashboard_alert_-unknown-_-unknown-,ai_show_data_translations,ai_update_cached_data_admin_templates,ai_listings_-unknown-_-unknown-,ai_life_-unknown-_-unknown-,ai_preapproval_message_post_message_post,ai_requested_view_p5,ai_show_view_view_listing,ai_office_location_-unknown-_-unknown-,ai_ask_question_submit_contact_host,ai_reviews_data_user_reviews,ai_currencies_nan_nan,ai_signup_weibo_referral_-unknown-_-unknown-,ai_why_host_-unknown-_-unknown-,ai_domains_-unknown-_-unknown-,ai_phone_verification_phone_number_removed_-unknown-_-unknown-,ai_social_connections_data_user_social_connections,ai_ajax_refresh_subtotal_click_change_trip_characteristics,ai_recent_reservations_-unknown-_-unknown-,ai_settings_-unknown-_-unknown-,ai_jumio_-unknown-_-unknown-,ai_feed_-unknown-_-unknown-,ai_localization_settings_nan_nan,ai_click_click_book_it,ai_detect_fb_session_-unknown-_-unknown-,ai_email_by_key_-unknown-_-unknown-,ai_review_page_-unknown-_-unknown-,ai_other_hosting_reviews_first_-unknown-_-unknown-,ai_complete_status_-unknown-_-unknown-,ai_read_policy_click_click_read_policy_click,ai_listings_view_user_listings,ai_recommendations_data_listing_recommendations,ai_pay_-unknown-_-unknown-,ai_referrer_status_-unknown-_-unknown-,ai_apply_coupon_error_click_apply_coupon_error,ai_new_-unknown-_-unknown-,ai_guest_booked_elsewhere_message_post_message_post,ai_photography_-unknown-_-unknown-,ai_webcam_upload_-unknown-_-unknown-,ai_personalize_data_wishlist_content_update,ai_signup_login_view_sign

time: 760 ms (started: 2021-08-18 18:45:03 +00:00)


In [22]:
tmp.drop('action_info', axis=1, inplace=True)

time: 831 ms (started: 2021-08-18 18:45:04 +00:00)


#### Checking counts of missing values per each column

In [23]:
# not_missing = pd.DataFrame(tmp.notna().sum()).reset_index()
# not_missing.columns = ['col', 'counts']
# not_missing['ratio'] = not_missing['counts'].apply(lambda x: round(x / len(users), 4))
# not_missing.shape

time: 483 µs (started: 2021-08-18 18:45:05 +00:00)


In [24]:
# not_missing.head()

time: 2.7 ms (started: 2021-08-18 18:45:05 +00:00)


In [25]:
# threshold = 0.00005
# mask = not_missing.ratio > threshold
# mask.sum()

time: 2.55 ms (started: 2021-08-18 18:45:05 +00:00)


#### Dropping all columns that are lower than the above threshold
Decided not to do that in this iteration

In [26]:
# keep_columns = not_missing[mask].col.tolist()
# len(keep_columns)

time: 309 µs (started: 2021-08-18 18:45:05 +00:00)


In [27]:
# keep_columns[0], keep_columns[-1]

time: 388 µs (started: 2021-08-18 18:45:05 +00:00)


### 2.1.c Saving features

In [28]:
# features1 = tmp[keep_columns].copy(deep=True)
features1 = tmp.copy(deep=True)
features1.shape

(135483, 674)

time: 248 ms (started: 2021-08-18 18:45:05 +00:00)


### 2.1.1 Count of each action_type normalized

In [29]:
col = 'action_type'
col_values = list(sessions[col].unique())
len(col_values)

11

time: 738 ms (started: 2021-08-18 18:45:05 +00:00)


In [30]:
tmp = sessions[['user_id', col]].groupby('user_id', as_index=False).agg(list)
tmp.shape

(135483, 2)

time: 5 s (started: 2021-08-18 18:45:06 +00:00)


In [31]:
tmp['size'] = tmp[col].apply(lambda x: len(x))

time: 117 ms (started: 2021-08-18 18:45:11 +00:00)


In [32]:
tmp['counts'] = tmp[col].apply(lambda x: dict(Counter(x)))

time: 887 ms (started: 2021-08-18 18:45:11 +00:00)


In [33]:
tmp.head()

,user_id,action_type,size,counts
0,00023iyk9l,"[partner_callback, booking_request, data, None...",40,"{'partner_callback': 1, 'booking_request': 1, ..."
1,0010k6l0om,"[partner_callback, click, None, view, None, No...",63,"{'partner_callback': 1, 'click': 16, None: 15,..."
2,001wyh0pz8,"[submit, click, click, -unknown-, -unknown-, c...",90,"{'submit': 3, 'click': 66, '-unknown-': 6, 'vi..."
3,0028jgx1x1,"[submit, view, view, data, view, data, view, d...",31,"{'submit': 1, 'view': 15, 'data': 5, '-unknown..."
4,002qnbzfs5,"[None, click, view, data, click, view, data, v...",789,"{None: 77, 'click': 140, 'view': 216, 'data': ..."


time: 16.3 ms (started: 2021-08-18 18:45:12 +00:00)


In [34]:
tmp = pd.concat([tmp, pd.json_normalize(tmp['counts'])], axis=1)

time: 1.34 s (started: 2021-08-18 18:45:12 +00:00)


In [35]:
tmp.drop(['action_type', 'counts'], axis=1, inplace=True)

time: 59.3 ms (started: 2021-08-18 18:45:13 +00:00)


In [36]:
tmp.head()

,user_id,size,partner_callback,booking_request,data,NaN,view,click,message_post,-unknown-,submit,modify,booking_response
0,00023iyk9l,40,1.00,1.00,9.00,3.00,21.00,4.00,1.00,NaN,NaN,NaN,NaN
1,0010k6l0om,63,1.00,NaN,9.00,15.00,17.00,16.00,NaN,5.00,NaN,NaN,NaN
2,001wyh0pz8,90,NaN,NaN,2.00,5.00,8.00,66.00,NaN,6.00,3.00,NaN,NaN
3,0028jgx1x1,31,NaN,NaN,5.00,NaN,15.00,9.00,NaN,1.00,1.00,NaN,NaN
4,002qnbzfs5,789,NaN,1.00,140.00,77.00,216.00,140.00,16.00,184.00,15.00,NaN,NaN


time: 37.1 ms (started: 2021-08-18 18:45:13 +00:00)


In [37]:
cols = list(tmp)[2:]
cols = [f'at_{e}' for e in cols]

time: 1.14 ms (started: 2021-08-18 18:45:14 +00:00)


In [38]:
tmp.columns = ['user_id', 'size'] + cols

time: 668 µs (started: 2021-08-18 18:45:14 +00:00)


In [39]:
for e in cols:
    tmp[e] = tmp[e] / tmp['size']

time: 27.1 ms (started: 2021-08-18 18:45:14 +00:00)


In [40]:
tmp.head()

,user_id,size,at_partner_callback,at_booking_request,at_data,at_None,at_view,at_click,at_message_post,at_-unknown-,at_submit,at_modify,at_booking_response
0,00023iyk9l,40,0.03,0.03,0.23,0.07,0.53,0.10,0.03,NaN,NaN,NaN,NaN
1,0010k6l0om,63,0.02,NaN,0.14,0.24,0.27,0.25,NaN,0.08,NaN,NaN,NaN
2,001wyh0pz8,90,NaN,NaN,0.02,0.06,0.09,0.73,NaN,0.07,0.03,NaN,NaN
3,0028jgx1x1,31,NaN,NaN,0.16,NaN,0.48,0.29,NaN,0.03,0.03,NaN,NaN
4,002qnbzfs5,789,NaN,0.00,0.18,0.10,0.27,0.18,0.02,0.23,0.02,NaN,NaN


time: 38.3 ms (started: 2021-08-18 18:45:14 +00:00)


In [41]:
tmp.drop(['size'], axis=1, inplace=True)

time: 14.8 ms (started: 2021-08-18 18:45:14 +00:00)


In [42]:
tmp.fillna(0, inplace=True)

time: 88.6 ms (started: 2021-08-18 18:45:14 +00:00)


In [43]:
tmp.head()

,user_id,at_partner_callback,at_booking_request,at_data,at_None,at_view,at_click,at_message_post,at_-unknown-,at_submit,at_modify,at_booking_response
0,00023iyk9l,0.03,0.03,0.23,0.07,0.53,0.10,0.03,0.00,0.00,0.00,0.00
1,0010k6l0om,0.02,0.00,0.14,0.24,0.27,0.25,0.00,0.08,0.00,0.00,0.00
2,001wyh0pz8,0.00,0.00,0.02,0.06,0.09,0.73,0.00,0.07,0.03,0.00,0.00
3,0028jgx1x1,0.00,0.00,0.16,0.00,0.48,0.29,0.00,0.03,0.03,0.00,0.00
4,002qnbzfs5,0.00,0.00,0.18,0.10,0.27,0.18,0.02,0.23,0.02,0.00,0.00


time: 21.3 ms (started: 2021-08-18 18:45:14 +00:00)


In [44]:
features1a = tmp.copy(deep=True)
features1a.shape

(135483, 12)

time: 16.1 ms (started: 2021-08-18 18:45:14 +00:00)


### 2.2 Generating features based on seconds elapsed and deltas between info

In [45]:
tmp = sessions[['user_id', 'secs_elapsed']].groupby('user_id', as_index=False).agg(list)
tmp.shape

(135483, 2)

time: 5.4 s (started: 2021-08-18 18:45:14 +00:00)


In [46]:
tmp.head()

,user_id,secs_elapsed
0,00023iyk9l,"[-1.0, 0.0, 6.0, 45.0, 81.0, 94.0, 112.0, 155...."
1,0010k6l0om,"[-1.0, 3.0, 9.0, 22.0, 26.0, 30.0, 34.0, 36.0,..."
2,001wyh0pz8,"[-1.0, 35.0, 80.0, 91.0, 108.0, 118.0, 142.0, ..."
3,0028jgx1x1,"[-1.0, 3.0, 5.0, 20.0, 28.0, 75.0, 86.0, 91.0,..."
4,002qnbzfs5,"[-1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,..."


time: 16.5 ms (started: 2021-08-18 18:45:19 +00:00)


In [47]:
tmp.secs_elapsed = tmp.secs_elapsed.apply(lambda x: [0] + x[1:])

time: 931 ms (started: 2021-08-18 18:45:19 +00:00)


In [48]:
tmp['deltas'] = tmp['secs_elapsed'].apply(lambda x: [int(j - i) for i, j in zip(x[:-1], x[1:])])

time: 2.43 s (started: 2021-08-18 18:45:20 +00:00)


In [49]:
tmp.head()

,user_id,secs_elapsed,deltas
0,00023iyk9l,"[0, 0.0, 6.0, 45.0, 81.0, 94.0, 112.0, 155.0, ...","[0, 6, 39, 36, 13, 18, 43, 8, 19, 187, 26, 15,..."
1,0010k6l0om,"[0, 3.0, 9.0, 22.0, 26.0, 30.0, 34.0, 36.0, 39...","[3, 6, 13, 4, 4, 4, 2, 3, 6, 1, 3, 4, 30, 8, 1..."
2,001wyh0pz8,"[0, 35.0, 80.0, 91.0, 108.0, 118.0, 142.0, 201...","[35, 45, 11, 17, 10, 24, 59, 1, 54, 100, 2, 9,..."
3,0028jgx1x1,"[0, 3.0, 5.0, 20.0, 28.0, 75.0, 86.0, 91.0, 97...","[3, 2, 15, 8, 47, 11, 5, 6, 2, 15, 13, 19, 76,..."
4,002qnbzfs5,"[0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


time: 20 ms (started: 2021-08-18 18:45:23 +00:00)


In [50]:
def get_statistics(x):
    if not x:
        return None, None, None, None
    x = np.array(x)
    return x.mean(), x.std(), x.max(), np.median(x)

time: 1.33 ms (started: 2021-08-18 18:45:23 +00:00)


In [51]:
def get_statistics_no_outliers(x):
    if not x:
        return None, None, None, None, None
    x = np.array(x)
    initial_size = len(x)
    x = [e for e in x if e <= x.mean() + x.std()]
    outliers_count = initial_size - len(x)
    x = np.array(x)
    return x.mean(), x.std(), x.max(), np.median(x), outliers_count

time: 2.27 ms (started: 2021-08-18 18:45:23 +00:00)


In [52]:
get_statistics(tmp.iloc[0].deltas)

(14542.692307692309, 69958.77759379552, 437348, 64.0)

time: 13 ms (started: 2021-08-18 18:45:23 +00:00)


In [53]:
get_statistics_no_outliers(tmp.iloc[0].deltas)

(1165.1351351351352, 2758.241156794633, 11029, 58.0, 2)

time: 12.9 ms (started: 2021-08-18 18:45:23 +00:00)


In [54]:
tmp = pd.concat([tmp, tmp.deltas.progress_apply(lambda x: pd.Series(get_statistics(x)))], axis=1)
tmp.shape

  0%|          | 0/135483 [00:00<?, ?it/s]

(135483, 7)

time: 1min 6s (started: 2021-08-18 18:45:23 +00:00)


In [55]:
tmp.columns = ['user_id', 'secs_elapsed', 'deltas', 'deltas_mean', 'deltas_std', 'deltas_max', 'deltas_median']

time: 916 µs (started: 2021-08-18 18:46:30 +00:00)


In [56]:
tmp.head()

,user_id,secs_elapsed,deltas,deltas_mean,deltas_std,deltas_max,deltas_median
0,00023iyk9l,"[0, 0.0, 6.0, 45.0, 81.0, 94.0, 112.0, 155.0, ...","[0, 6, 39, 36, 13, 18, 43, 8, 19, 187, 26, 15,...",14542.69,69958.78,437348.00,64.00
1,0010k6l0om,"[0, 3.0, 9.0, 22.0, 26.0, 30.0, 34.0, 36.0, 39...","[3, 6, 13, 4, 4, 4, 2, 3, 6, 1, 3, 4, 30, 8, 1...",2062.87,6002.06,34874.00,38.50
2,001wyh0pz8,"[0, 35.0, 80.0, 91.0, 108.0, 118.0, 142.0, 201...","[35, 45, 11, 17, 10, 24, 59, 1, 54, 100, 2, 9,...",567.96,3206.52,30047.00,33.00
3,0028jgx1x1,"[0, 3.0, 5.0, 20.0, 28.0, 75.0, 86.0, 91.0, 97...","[3, 2, 15, 8, 47, 11, 5, 6, 2, 15, 13, 19, 76,...",2821.20,7456.50,37388.00,170.00
4,002qnbzfs5,"[0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",1799.85,34343.17,946465.00,3.00


time: 39 ms (started: 2021-08-18 18:46:30 +00:00)


In [57]:
tmp = pd.concat([tmp, tmp.deltas.progress_apply(lambda x: pd.Series(get_statistics_no_outliers(x)))], axis=1)
tmp.shape

  0%|          | 0/135483 [00:00<?, ?it/s]

(135483, 12)

time: 16min 18s (started: 2021-08-18 18:46:30 +00:00)


In [58]:
tmp.columns = [
    'user_id', 'secs_elapsed', 'deltas', 'deltas_mean', 'deltas_std', 'deltas_max', 'deltas_median', 
    'deltas_no_mean', 'deltas_no_std', 'deltas_no_max', 'deltas_no_median', 'deltas_no_num_outliers'
]

time: 889 µs (started: 2021-08-18 19:02:47 +00:00)


In [59]:
tmp.head()

,user_id,secs_elapsed,deltas,deltas_mean,deltas_std,deltas_max,deltas_median,deltas_no_mean,deltas_no_std,deltas_no_max,deltas_no_median,deltas_no_num_outliers
0,00023iyk9l,"[0, 0.0, 6.0, 45.0, 81.0, 94.0, 112.0, 155.0, ...","[0, 6, 39, 36, 13, 18, 43, 8, 19, 187, 26, 15,...",14542.69,69958.78,437348.00,64.00,1165.14,2758.24,11029.00,58.00,2.00
1,0010k6l0om,"[0, 3.0, 9.0, 22.0, 26.0, 30.0, 34.0, 36.0, 39...","[3, 6, 13, 4, 4, 4, 2, 3, 6, 1, 3, 4, 30, 8, 1...",2062.87,6002.06,34874.00,38.50,692.14,1599.51,8000.00,27.00,4.00
2,001wyh0pz8,"[0, 35.0, 80.0, 91.0, 108.0, 118.0, 142.0, 201...","[35, 45, 11, 17, 10, 24, 59, 1, 54, 100, 2, 9,...",567.96,3206.52,30047.00,33.00,189.75,501.59,3212.00,33.00,2.00
3,0028jgx1x1,"[0, 3.0, 5.0, 20.0, 28.0, 75.0, 86.0, 91.0, 97...","[3, 2, 15, 8, 47, 11, 5, 6, 2, 15, 13, 19, 76,...",2821.20,7456.50,37388.00,170.00,988.00,1871.10,9313.00,105.00,2.00
4,002qnbzfs5,"[0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",1799.85,34343.17,946465.00,3.00,235.77,1650.94,24728.00,3.00,4.00


time: 35.4 ms (started: 2021-08-18 19:02:47 +00:00)


### 2.2.1 Adding stats on seconds elapsed

In [60]:
tmp = pd.concat([tmp, tmp.secs_elapsed.progress_apply(lambda x: pd.Series(get_statistics(x)))], axis=1)
tmp.shape

  0%|          | 0/135483 [00:00<?, ?it/s]

(135483, 16)

time: 1min 14s (started: 2021-08-18 19:02:47 +00:00)


In [61]:
tmp.columns = [
    'user_id', 'secs_elapsed', 'deltas', 'deltas_mean', 'deltas_std', 'deltas_max', 'deltas_median', 
    'deltas_no_mean', 'deltas_no_std', 'deltas_no_max', 'deltas_no_median', 'deltas_no_num_outliers',
    'secs_elapsed_mean', 'secs_elapsed_std', 'secs_elapsed_max', 'secs_elapsed_median',
]

time: 1.41 ms (started: 2021-08-18 19:04:02 +00:00)


In [62]:
tmp.drop(['secs_elapsed', 'deltas'], axis=1, inplace=True)

time: 66.6 ms (started: 2021-08-18 19:04:02 +00:00)


In [63]:
features2 = tmp.copy(deep=True)
features2.shape

(135483, 14)

time: 19.1 ms (started: 2021-08-18 19:04:02 +00:00)


### 2.3 Generating features based on device type info

In [64]:
tmp = sessions[['user_id', 'device_type']].groupby('user_id', as_index=False).agg(set)
tmp.shape

(135483, 2)

time: 4.75 s (started: 2021-08-18 19:04:02 +00:00)


In [65]:
tmp['size'] = tmp.device_type.apply(lambda x: len(x))

time: 115 ms (started: 2021-08-18 19:04:07 +00:00)


In [66]:
tmp.drop('device_type', axis=1, inplace=True)

time: 43.8 ms (started: 2021-08-18 19:04:07 +00:00)


In [67]:
tmp.head()

,user_id,size
0,00023iyk9l,2
1,0010k6l0om,1
2,001wyh0pz8,1
3,0028jgx1x1,2
4,002qnbzfs5,2


time: 18.2 ms (started: 2021-08-18 19:04:07 +00:00)


In [68]:
tmp.columns = ['user_id', 'device_count']

time: 902 µs (started: 2021-08-18 19:04:07 +00:00)


In [69]:
tmp.head()

,user_id,device_count
0,00023iyk9l,2
1,0010k6l0om,1
2,001wyh0pz8,1
3,0028jgx1x1,2
4,002qnbzfs5,2


time: 16.4 ms (started: 2021-08-18 19:04:07 +00:00)


In [70]:
features3 = tmp.copy(deep=True)
features3.shape

(135483, 2)

time: 7.72 ms (started: 2021-08-18 19:04:07 +00:00)


### 3.1 Features based on Users table

In [71]:
users['dow_registered'] = users.date_account_created.dt.weekday
users['day_registered'] = users.date_account_created.dt.day
users['month_registered'] = users.date_account_created.dt.month
users['year_registered'] = users.date_account_created.dt.year

time: 166 ms (started: 2021-08-18 19:04:08 +00:00)


In [72]:
users['hr_registered'] = users.timestamp_first_active.dt.hour

time: 36.9 ms (started: 2021-08-18 19:04:08 +00:00)


In [73]:
users.age.max()

2014.0

time: 17.4 ms (started: 2021-08-18 19:04:08 +00:00)


In [74]:
users.head()

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,train_flag,dow_registered,day_registered,month_registered,year_registered,hr_registered
0,gxn3p5htnn,2010-06-28,2009-03-19 04:32:55,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF,1,0,28,6,2010,4
1,820tgsjxq7,2011-05-25,2009-05-23 17:48:09,MALE,38.00,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF,1,2,25,5,2011,17
2,4ft3gnwmtx,2010-09-28,2009-06-09 23:12:47,FEMALE,56.00,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US,1,1,28,9,2010,23
3,bjjt8pjhuk,2011-12-05,2009-10-31 06:01:29,FEMALE,42.00,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other,1,0,5,12,2011,6
4,87mebub9p4,2010-09-14,2009-12-08 06:11:05,-unknown-,41.00,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US,1,1,14,9,2010,6


time: 43.3 ms (started: 2021-08-18 19:04:08 +00:00)


In [75]:
mask = (users.age > 1000) & (users.age < 2000)
users.loc[mask, 'age'] = 2015 - users.loc[mask, 'age']
mask.sum()

71

time: 20.3 ms (started: 2021-08-18 19:04:08 +00:00)


In [76]:
users.loc[(users['age'] > 105) | (users['age'] < 14), 'age'] = -1
users['age'].fillna(-1, inplace=True)

time: 18.2 ms (started: 2021-08-18 19:04:08 +00:00)


In [77]:
bins = [-1, 20, 25, 30, 40, 50, 60, 75, 85, 105]
users['age_group'] = np.digitize(users['age'], bins, right=True)

time: 25.9 ms (started: 2021-08-18 19:04:08 +00:00)


In [78]:
users.sample(5)

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,train_flag,dow_registered,day_registered,month_registered,year_registered,hr_registered,age_group
71413,vwb4kjvpo3,2013-04-17,2013-04-17 04:34:54,-unknown-,-1.00,basic,0,en,other,other,linked,Web,Mac Desktop,Safari,NDF,1,2,17,4,2013,4,0
93932,d1ojm9m718,2013-07-25,2013-07-25 16:44:18,FEMALE,29.00,basic,25,en,direct,direct,untracked,iOS,iPhone,-unknown-,other,1,3,25,7,2013,16,3
110353,m9gb3wtmgs,2013-09-23,2013-09-23 20:13:26,-unknown-,-1.00,basic,0,en,direct,direct,untracked,Web,Windows Desktop,Firefox,NDF,1,0,23,9,2013,20,0
116816,vw754ou9v4,2013-10-14,2013-10-14 21:52:11,-unknown-,31.00,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,US,1,0,14,10,2013,21,4
118992,64j2hbzi4j,2013-10-23,2013-10-23 17:54:46,MALE,38.00,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Safari,NDF,1,2,23,10,2013,17,4


time: 51.5 ms (started: 2021-08-18 19:04:08 +00:00)


In [79]:
users.shape

(275547, 22)

time: 3.39 ms (started: 2021-08-18 19:04:08 +00:00)


### 3.1.1. Dropping redundand columns

In [80]:
users.drop(['date_account_created', 'timestamp_first_active'], axis=1, inplace=True)

time: 56.7 ms (started: 2021-08-18 19:04:08 +00:00)


In [81]:
users.columns = ['user_id'] + list(users)[1:]

time: 3.49 ms (started: 2021-08-18 19:04:08 +00:00)


In [82]:
users.head()

,user_id,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,train_flag,dow_registered,day_registered,month_registered,year_registered,hr_registered,age_group
0,gxn3p5htnn,-unknown-,-1.00,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF,1,0,28,6,2010,4,0
1,820tgsjxq7,MALE,38.00,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF,1,2,25,5,2011,17,4
2,4ft3gnwmtx,FEMALE,56.00,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US,1,1,28,9,2010,23,6
3,bjjt8pjhuk,FEMALE,42.00,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other,1,0,5,12,2011,6,5
4,87mebub9p4,-unknown-,41.00,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US,1,1,14,9,2010,6,5


time: 77.7 ms (started: 2021-08-18 19:04:08 +00:00)


In [83]:
users.shape

(275547, 20)

time: 7.73 ms (started: 2021-08-18 19:04:08 +00:00)


#### 4. Assembling all features into one dataset

In [84]:
df = users.merge(features1, on='user_id', how='left')
df.shape

(275547, 693)

time: 3.17 s (started: 2021-08-18 19:04:08 +00:00)


In [85]:
df = df.merge(features1a, on='user_id', how='left')
df.shape

(275547, 704)

time: 943 ms (started: 2021-08-18 19:04:11 +00:00)


In [86]:
df = df.merge(features2, on='user_id', how='left')
df.shape

(275547, 717)

time: 1.11 s (started: 2021-08-18 19:04:12 +00:00)


In [87]:
df = df.merge(features3, on='user_id', how='left')
df.shape

(275547, 718)

time: 966 ms (started: 2021-08-18 19:04:13 +00:00)


In [88]:
df.to_parquet('../data/processed/features.parquet')

time: 5.25 s (started: 2021-08-18 19:04:14 +00:00)


### 4.1 Splitting into train and test features

In [89]:
train_features = df[df.train_flag == 1]
train_features.shape

(213451, 718)

time: 4.15 s (started: 2021-08-18 19:04:20 +00:00)


In [90]:
train_features.to_parquet('../data/processed/train_features.parquet')

time: 4.7 s (started: 2021-08-18 19:04:24 +00:00)


In [91]:
test_features = df[df.train_flag == 0]
test_features.shape

(62096, 718)

time: 239 ms (started: 2021-08-18 19:04:29 +00:00)


In [92]:
test_features.to_parquet('../data/processed/test_features.parquet')

time: 2.93 s (started: 2021-08-18 19:04:29 +00:00)
